What this notebook does (Classical Baselines)

This notebook builds classical ML baselines for PLAsTiCC (SNIa vs SNII) to benchmark performance against the quantum classifier.
	1.	Load engineered features
	•	Reads ../data/plasticc/transient_features.csv (one row per transient + 16 engineered features + label).
	2.	Prepare labels + feature matrix
	•	Uses 16 engineered features.
	•	Encodes labels: SNII → 0, SNIa → 1.
	3.	Train/test split (leakage-safe)
	•	Splits once into 80% train / 20% test with stratification.
	•	All preprocessing decisions that require “fitting” are done on train only (via pipelines or explicit train-fit steps).
	4.	Train classical baselines (16 features)
	•	Logistic Regression (StandardScaler inside pipeline)
	•	Random Forest (no scaling required)
	•	CatBoost (optional, if installed)
	•	Soft Voting Ensemble (combines the above using predicted probabilities)
	5.	Evaluate + save results
	•	Reports Accuracy, Balanced Accuracy, ROC AUC, and confusion matrices.
	•	Saves metrics to ../results/plasticc_classical_results_2k.json.
	6.	Top-3 “apples-to-apples” baseline (optional section)
	•	Selects the top 3 features (same criterion as quantum) using TRAIN only.
	•	Runs the same classical models using only those 3 features.
	•	Saves results to ../results/plasticc_classical_top3_results.json.



In [1]:
# ============================================================
# 02_classical_with_16_features.ipynb  (FIXED)
# ============================================================

import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    roc_auc_score,
)

# CatBoost optional (but recommended)
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except Exception as e:
    CATBOOST_AVAILABLE = False
    print("⚠️ CatBoost not available. Install with: pip install catboost")
    print("   Error:", e)

RANDOM_STATE = 42
TEST_SIZE = 0.2

DATA_DIR = os.path.join("..", "data", "plasticc")
FEATURES_PATH = os.path.join(DATA_DIR, "transient_features.csv")

RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

OUT_RESULTS_JSON = os.path.join(RESULTS_DIR, "plasticc_classical_results_2k.json")
OUT_FEATURE_SELECTION_JSON = os.path.join(RESULTS_DIR, "top_3_features.json")
OUT_CORR_CSV = os.path.join(RESULTS_DIR, "feature_correlations_train_only.csv")

feature_cols = [
    "mag_min", "mag_max", "mag_mean", "mag_std", "mag_range",
    "flux_max", "flux_mean", "flux_std",
    "time_span", "rise_time", "decline_time", "rise_decline_ratio",
    "mean_rise_slope", "mean_decline_slope", "max_slope",
    "n_points"
]

label_map = {"SNII": 0, "SNIa": 1}

# -----------------------------
# 1) Load
# -----------------------------
df = pd.read_csv(FEATURES_PATH)

missing = [c for c in feature_cols + ["label"] if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

X = df[feature_cols].values
y = df["label"].map(label_map).values
if np.any(pd.isnull(y)):
    bad = df.loc[pd.isnull(y), "label"].unique()
    raise ValueError(f"Unknown labels found: {bad}")

print(f"Loaded {len(df)} samples with {len(df.columns)} columns")
print("\nClass distribution:")
print(df["label"].value_counts())
print(f"\nFeature matrix: {X.shape}")
print(f"Labels: {y.shape}, distribution: {np.unique(y, return_counts=True)}")

# -----------------------------
# 2) Train/test split (ONCE)
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"\nTrain: {len(X_train)}, Test: {len(X_test)}")

# -----------------------------
# 3) Models (pipelines to avoid wrong scaling)
# -----------------------------
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=3000, C=2.0, random_state=RANDOM_STATE))
])

rf = RandomForestClassifier(
    n_estimators=600,
    max_depth=20,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

if CATBOOST_AVAILABLE:
    cb = CatBoostClassifier(
        iterations=800,
        depth=8,
        learning_rate=0.05,
        random_seed=RANDOM_STATE,
        loss_function="Logloss",
        eval_metric="AUC",
        verbose=False
    )

models = {
    "Logistic Regression": lr_pipe,
    "Random Forest": rf,
}

if CATBOOST_AVAILABLE:
    models["CatBoost"] = cb

# Ensemble: all estimators receive raw X; pipelines handle preprocessing internally
estimators = [("lr", lr_pipe), ("rf", rf)]
weights = [1, 2]
if CATBOOST_AVAILABLE:
    estimators.append(("cb", cb))
    weights.append(3)

ensemble = VotingClassifier(
    estimators=estimators,
    voting="soft",
    weights=weights,
    n_jobs=-1
)
models["Ensemble"] = ensemble

# -----------------------------
# 4) Train + Evaluate
# -----------------------------
print("\n" + "=" * 70)
print("TRAINING + EVALUATION (CLASSICAL)")
print("=" * 70)

results = {}

for name, model in models.items():
    print(f"\n{name}")
    print("-" * 50)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # probability for AUC
    proba = model.predict_proba(X_test)
    if proba.ndim == 2 and proba.shape[1] == 2:
        scores = proba[:, 1]
    else:
        scores = np.ravel(proba)

    acc = accuracy_score(y_test, y_pred)
    bacc = balanced_accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, scores)

    cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    print(f"Accuracy:          {acc:.3f}")
    print(f"Balanced accuracy: {bacc:.3f}")
    print(f"AUC:               {auc:.3f}")
    print("Confusion matrix [SNII, SNIa]:")
    print(cm)

    results[name] = {
        "accuracy": float(acc),
        "balanced_accuracy": float(bacc),
        "auc": float(auc),
        "confusion_matrix": cm.tolist(),
        "tn_fp_fn_tp": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    }

with open(OUT_RESULTS_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Results saved: {OUT_RESULTS_JSON}")

# -----------------------------
# 5) Feature selection for quantum (TRAIN ONLY, no leakage)
# -----------------------------
from scipy.stats import pointbiserialr

corr_rows = []
for i, feat in enumerate(feature_cols):
    corr, pval = pointbiserialr(y_train, X_train[:, i])
    corr_rows.append({
        "feature": feat,
        "abs_corr": float(abs(corr)),
        "signed_corr": float(corr),
        "p_value": float(pval),
    })

corr_df = pd.DataFrame(corr_rows).sort_values("abs_corr", ascending=False)
top_3_features = corr_df.head(3)["feature"].tolist()

corr_df.to_csv(OUT_CORR_CSV, index=False)
with open(OUT_FEATURE_SELECTION_JSON, "w") as f:
    json.dump(
        {
            "top_3_features": top_3_features,
            "method": "top3_by_abs_pointbiserial_correlation_train_only",
            "random_state": RANDOM_STATE,
            "test_size": TEST_SIZE,
        },
        f,
        indent=2
    )

print("\n" + "=" * 70)
print("FEATURE SELECTION (TRAIN ONLY)")
print("=" * 70)
print("Top 10 correlations (train only):")
print(corr_df.head(10).to_string(index=False))
print("\n🎯 TOP 3 FEATURES FOR QUANTUM:", top_3_features)
print(f"\n✓ Saved: {OUT_FEATURE_SELECTION_JSON}")
print(f"✓ Saved: {OUT_CORR_CSV}")

Loaded 1072 samples with 18 columns

Class distribution:
label
SNII    549
SNIa    523
Name: count, dtype: int64

Feature matrix: (1072, 16)
Labels: (1072,), distribution: (array([0, 1]), array([549, 523]))

Train: 857, Test: 215

TRAINING + EVALUATION (CLASSICAL)

Logistic Regression
--------------------------------------------------
Accuracy:          0.712
Balanced accuracy: 0.712
AUC:               0.770
Confusion matrix [SNII, SNIa]:
[[75 35]
 [27 78]]

Random Forest
--------------------------------------------------
Accuracy:          0.758
Balanced accuracy: 0.759
AUC:               0.845
Confusion matrix [SNII, SNIa]:
[[81 29]
 [23 82]]

CatBoost
--------------------------------------------------
Accuracy:          0.744
Balanced accuracy: 0.744
AUC:               0.846
Confusion matrix [SNII, SNIa]:
[[83 27]
 [28 77]]

Ensemble
--------------------------------------------------
Accuracy:          0.744
Balanced accuracy: 0.745
AUC:               0.853
Confusion matrix [SNII, S

In [3]:
# ============================================================
# Classical Top-3 Baseline (LEAKAGE-SAFE, apples-to-apples with quantum)
# - Split FIRST (train/test)
# - Select top 3 features using TRAIN ONLY (point-biserial correlation)
# - Fit preprocessing on TRAIN ONLY (NaN/Inf clean -> clip p1/p99 -> optional log)
# - Train models: LR (scaled via Pipeline), RF, CatBoost (optional), Soft Voting Ensemble
# - Save: ../results/plasticc_classical_top3_results.json
#        ../results/top_3_features_train_only.json
#        ../results/feature_correlations_train_only_top3.csv
# ============================================================

import os
import json
import numpy as np
import pandas as pd

from scipy.stats import pointbiserialr

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

# CatBoost optional
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except Exception as e:
    CATBOOST_AVAILABLE = False
    print("⚠️ CatBoost not available. Install with: pip install catboost")
    print("   Error:", e)

# -----------------------------
# 0) Config
# -----------------------------
RANDOM_STATE = 42
TEST_SIZE = 0.2

DATA_DIR = os.path.join("..", "data", "plasticc")
FEATURES_PATH = os.path.join(DATA_DIR, "transient_features.csv")

RESULTS_DIR = os.path.join("..", "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

OUT_JSON = os.path.join(RESULTS_DIR, "plasticc_classical_top3_results.json")
OUT_TOP3_JSON = os.path.join(RESULTS_DIR, "top_3_features_train_only.json")
OUT_CORR_CSV = os.path.join(RESULTS_DIR, "feature_correlations_train_only_top3.csv")

label_map = {"SNII": 0, "SNIa": 1}

feature_cols = [
    "mag_min", "mag_max", "mag_mean", "mag_std", "mag_range",
    "flux_max", "flux_mean", "flux_std",
    "time_span", "rise_time", "decline_time", "rise_decline_ratio",
    "mean_rise_slope", "mean_decline_slope", "max_slope",
    "n_points"
]

print("=" * 70)
print("Classical ML — PLAsTiCC (Top-3 features, leakage-safe)")
print("=" * 70)
print("FEATURES_PATH:", FEATURES_PATH)
print("RESULTS_DIR:", RESULTS_DIR)

# -----------------------------
# 1) Load data
# -----------------------------
df = pd.read_csv(FEATURES_PATH)

missing = [c for c in feature_cols + ["label"] if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

X_full = df[feature_cols].values
y = df["label"].map(label_map).values
if np.any(pd.isnull(y)):
    bad = df.loc[pd.isnull(y), "label"].unique()
    raise ValueError(f"Unknown labels found: {bad}")

print("\nLoaded samples:", len(df))
print("Label distribution:\n", df["label"].value_counts())
print("X_full shape:", X_full.shape)

# -----------------------------
# 2) Split FIRST (no leakage)
# -----------------------------
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_full, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

print("\nTrain full:", X_train_full.shape, "Test full:", X_test_full.shape)
print("Train label counts:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Test  label counts:", dict(zip(*np.unique(y_test, return_counts=True))))

# -----------------------------
# 3) Select top 3 features using TRAIN ONLY
# -----------------------------
corr_rows = []
for i, feat in enumerate(feature_cols):
    corr, pval = pointbiserialr(y_train, X_train_full[:, i])
    corr_rows.append({
        "feature": feat,
        "abs_corr": float(abs(corr)),
        "signed_corr": float(corr),
        "p_value": float(pval),
    })

corr_df = pd.DataFrame(corr_rows).sort_values("abs_corr", ascending=False)
selected_features = corr_df.head(3)["feature"].tolist()
selected_idx = [feature_cols.index(f) for f in selected_features]

corr_df.to_csv(OUT_CORR_CSV, index=False)
with open(OUT_TOP3_JSON, "w") as f:
    json.dump(
        {
            "top_3_features": selected_features,
            "method": "top3_by_abs_pointbiserial_correlation_train_only",
            "random_state": RANDOM_STATE,
            "test_size": TEST_SIZE,
        },
        f,
        indent=2
    )

print("\nTop 10 correlations (TRAIN only):")
print(corr_df.head(10).to_string(index=False))
print("\n🎯 Selected top 3 (TRAIN only):", selected_features)
print(f"✓ Saved: {OUT_TOP3_JSON}")
print(f"✓ Saved: {OUT_CORR_CSV}")

# Slice train/test to selected features
X_train = X_train_full[:, selected_idx].copy()
X_test  = X_test_full[:,  selected_idx].copy()

# -----------------------------
# 4) Preprocessing fit on TRAIN only, apply to TEST
#    - NaN/Inf clean
#    - clip p1/p99 from TRAIN
#    - decide log-transform from TRAIN
# -----------------------------
def clean_clip_log_fit_transform(X_tr, X_te, feature_names):
    # Clean NaN/Inf
    if np.any(np.isnan(X_tr)) or np.any(np.isinf(X_tr)):
        print("\n⚠️ Found NaN/Inf in TRAIN — cleaning with nan_to_num.")
        X_tr = np.nan_to_num(X_tr, nan=0.0, posinf=1e10, neginf=-1e10)

    if np.any(np.isnan(X_te)) or np.any(np.isinf(X_te)):
        print("\n⚠️ Found NaN/Inf in TEST — cleaning with nan_to_num.")
        X_te = np.nan_to_num(X_te, nan=0.0, posinf=1e10, neginf=-1e10)

    X_tr2 = X_tr.copy()
    X_te2 = X_te.copy()

    clip_bounds = []
    log_transformed = []
    log_shifts = {}

    for j in range(X_tr2.shape[1]):
        # Clip bounds from TRAIN only
        p1, p99 = np.percentile(X_tr2[:, j], [1, 99])
        X_tr2[:, j] = np.clip(X_tr2[:, j], p1, p99)
        X_te2[:, j] = np.clip(X_te2[:, j], p1, p99)
        clip_bounds.append((float(p1), float(p99)))

        # Decide log-transform from TRAIN only (same rule as quantum notebook)
        fmin, fmax = X_tr2[:, j].min(), X_tr2[:, j].max()
        dynamic_range = (fmax - fmin) / (abs(fmin) + 1e-10)

        if dynamic_range > 100:
            shift = float(X_tr2[:, j].min())  # TRAIN-only shift
            X_tr2[:, j] = np.log1p(X_tr2[:, j] - shift + 1)
            X_te2[:, j] = np.log1p(X_te2[:, j] - shift + 1)
            log_transformed.append(feature_names[j])
            log_shifts[feature_names[j]] = shift

    return X_tr2, X_te2, clip_bounds, log_transformed, log_shifts

X_train_proc, X_test_proc, clip_bounds, log_transformed, log_shifts = clean_clip_log_fit_transform(
    X_train, X_test, selected_features
)

print("\nClip bounds (TRAIN p1, p99) per feature:")
for feat, (p1, p99) in zip(selected_features, clip_bounds):
    print(f"  {feat}: [{p1:.4f}, {p99:.4f}]")
print("Log-transformed features:", log_transformed if log_transformed else "None")
if log_transformed:
    print("Log shifts (TRAIN min after clipping):", log_shifts)

# -----------------------------
# 5) Models
# -----------------------------
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=2000, C=2.0, random_state=RANDOM_STATE))
])

rf = RandomForestClassifier(
    n_estimators=600,
    max_depth=20,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

models = {
    "Logistic Regression": lr_pipe,
    "Random Forest": rf,
}

if CATBOOST_AVAILABLE:
    cb = CatBoostClassifier(
        iterations=800,
        depth=8,
        learning_rate=0.05,
        random_seed=RANDOM_STATE,
        loss_function="Logloss",
        eval_metric="AUC",
        verbose=False
    )
    models["CatBoost"] = cb

# Soft voting ensemble (pipelines handle their own preprocessing)
estimators = [("lr", lr_pipe), ("rf", rf)]
weights = [1, 2]
if CATBOOST_AVAILABLE:
    estimators.append(("cb", cb))
    weights.append(3)

ensemble = VotingClassifier(
    estimators=estimators,
    voting="soft",
    weights=weights,
    n_jobs=-1
)
models["Ensemble"] = ensemble

# -----------------------------
# 6) Train + Evaluate
# -----------------------------
def get_scores(model, X):
    proba = model.predict_proba(X)
    if proba.ndim == 2 and proba.shape[1] == 2:
        return proba[:, 1]
    return np.ravel(proba)

print("\n" + "=" * 70)
print("TRAINING + EVALUATION (CLASSICAL TOP-3, leakage-safe)")
print("=" * 70)

results = {
    "dataset_size": int(len(df)),
    "train_size": int(len(X_train_proc)),
    "test_size": int(len(X_test_proc)),
    "features_selected_auto": selected_features,
    "feature_selection_method": "top3_by_abs_pointbiserial_correlation_train_only",
    "preprocessing": {
        "order": "split -> select_features(train) -> fit_preproc(train) -> apply(test)",
        "outlier_clipping": "1st/99th percentile (fit on TRAIN)",
        "clip_bounds": {
            feat: {"p1": float(p1), "p99": float(p99)}
            for feat, (p1, p99) in zip(selected_features, clip_bounds)
        },
        "log_transformed_features": log_transformed,
        "log_shifts_train": log_shifts,
        "scaling": "StandardScaler inside LR pipeline; RF/CatBoost use raw preprocessed features",
    },
    "models": {}
}

for name, model in models.items():
    print(f"\n{name}")
    print("-" * 50)

    model.fit(X_train_proc, y_train)
    y_pred = model.predict(X_test_proc)
    scores = get_scores(model, X_test_proc)

    acc = accuracy_score(y_test, y_pred)
    bacc = balanced_accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, scores)

    cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    print(f"Accuracy:          {acc:.3f}")
    print(f"Balanced accuracy: {bacc:.3f}")
    print(f"AUC:               {auc:.3f}")
    print("Confusion matrix [SNII, SNIa]:")
    print(cm)
    print("\nClassification report:")
    print(classification_report(y_test, y_pred, target_names=["SNII", "SNIa"], zero_division=0))

    results["models"][name] = {
        "accuracy": float(acc),
        "balanced_accuracy": float(bacc),
        "auc": float(auc),
        "confusion_matrix": cm.tolist(),
        "tn_fp_fn_tp": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    }

# -----------------------------
# 7) Save results JSON
# -----------------------------
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Saved results to: {OUT_JSON}")

Classical ML — PLAsTiCC (Top-3 features, leakage-safe)
FEATURES_PATH: ../data/plasticc/transient_features.csv
RESULTS_DIR: ../results

Loaded samples: 1072
Label distribution:
 label
SNII    549
SNIa    523
Name: count, dtype: int64
X_full shape: (1072, 16)

Train full: (857, 16) Test full: (215, 16)
Train label counts: {np.int64(0): np.int64(439), np.int64(1): np.int64(418)}
Test  label counts: {np.int64(0): np.int64(110), np.int64(1): np.int64(105)}

Top 10 correlations (TRAIN only):
           feature  abs_corr  signed_corr      p_value
         time_span  0.279160    -0.279160 8.329884e-17
      decline_time  0.265730    -0.265730 2.565207e-15
           mag_max  0.160794     0.160794 2.232677e-06
           mag_std  0.150796     0.150796 9.275654e-06
          mag_mean  0.126317     0.126317 2.094389e-04
         mag_range  0.123106     0.123106 3.034252e-04
         rise_time  0.104246    -0.104246 2.245916e-03
mean_decline_slope  0.068830     0.068830 4.396452e-02
         max_s